<a href="https://colab.research.google.com/github/anish1108/pythonAutomation/blob/main/Python_%26_Automation_Intern_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Load the data
df = pd.read_csv("/content/raw_data.csv")  # adjust filename if needed

# Remove duplicate emails
df = df.drop_duplicates(subset="email")

# Normalize has_joined_event (Yes/No → True/False)
df["has_joined_event"] = df["has_joined_event"].str.strip().str.lower().map({
    "yes": True,
    "no": False
})

# Flag missing or incomplete LinkedIn profiles
def is_linkedin_missing(link):
    if pd.isna(link) or len(str(link).strip()) < 10 or "linkedin.com" not in link:
        return "Missing"
    return "OK"

df["linkedin_flag"] = df["What is your LinkedIn profile?"].apply(is_linkedin_missing)

# Flag missing job titles
df["job_title_flag"] = df["Job Title"].apply(lambda x: "Missing" if pd.isna(x) or str(x).strip() == "" else "OK")

# Save cleaned data
df.to_csv("/content/cleaned_output.csv", index=False)

print("✅ Data cleaned and saved as 'cleaned_output.csv'")


✅ Data cleaned and saved as 'cleaned_output.csv'


In [3]:
import pandas as pd
import json

# Load the cleaned data
df = pd.read_csv("/content/cleaned_output.csv")

# Store messages
messages = []

# Generate messages for each row
for _, row in df.iterrows():
    # Name logic: use first_name if available, else fallback to name
    name = row['first_name'] if pd.notna(row['first_name']) else row['name']
    job = row['Job Title'] if pd.notna(row['Job Title']) else "professional"
    joined = row['has_joined_event']
    linkedin_missing = row['linkedin_flag'] == 'Missing'

    # Generate the message
    if joined:
        msg = f"Hey {name}, thanks for joining our session! As a {job.lower()}, we think you’ll love our upcoming AI workflow tools. Want early access?"
    else:
        msg = f"Hi {name}, sorry we missed you at the last event! We’re preparing another session that might better suit your interests as a {job.lower()}."

    # Add LinkedIn encouragement if profile is missing
    if linkedin_missing:
        msg += " (P.S. We’d love to connect on LinkedIn!)"

    # Store the message
    messages.append({
        "email": row['email'],
        "message": msg
    })

# Save as CSV
output_df = pd.DataFrame(messages)
output_df.to_csv("/content/messages_output.csv", index=False)

# Also save as JSON (optional bonus)
with open("/content/messages_output.json", "w") as f:
    json.dump(messages, f, indent=2)

print("✅ Messages generated and saved as 'messages_output.csv' and 'messages_output.json'")


✅ Messages generated and saved as 'messages_output.csv' and 'messages_output.json'


In [4]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd

# Load generated messages
df = pd.read_csv("/content/messages_output.csv")

# Your email credentials
sender_email = "anishprasad2512@gmail.com"
app_password = "okyqbjusdqrdqwlp"

# Set up SMTP server
server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
server.login(sender_email, app_password)

sent_count = 0

for _, row in df.iterrows():
    recipient = row['email']
    message_text = row['message']

    # Construct the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient
    msg['Subject'] = "Thanks for engaging with our event!"

    msg.attach(MIMEText(message_text, 'plain'))

    try:
        server.sendmail(sender_email, recipient, msg.as_string())
        sent_count += 1
    except Exception as e:
        print(f"❌ Failed to send to {recipient}: {e}")

server.quit()
print(f"✅ Sent {sent_count} emails successfully.")


✅ Sent 71 emails successfully.
